In [1]:
!pip install accelerate

In [2]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP_proj_lzp'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['data', 'cot.ipynb', 'main.ipynb', 'aware_infer (1).ipynb', 'aware_infer.ipynb']


In [3]:
DATA_PATH = os.path.join(GOOGLE_DRIVE_PATH, 'data')
print(os.listdir(DATA_PATH))
file_name = 'train.json'
file_path = os.path.join(DATA_PATH, file_name)
print(file_path)

['test.json', 'test.tok.json', 'train.tok.json', 'train.json', 'train.csv', 'predict.csv']
drive/My Drive/NLP_proj_lzp/data/train.json


In [4]:
import re
import copy
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.generation import GenerationConfig
import json
import torch
import accelerate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")


tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.1")
model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-7b-v1.1", device_map=device)
model.generation_config = GenerationConfig.from_pretrained("lmsys/vicuna-7b-v1.1")
model.generation_config.do_sample = False

pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Conservative Sampling
hyperparameters_1 = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}

# Middleground Sampling
hyperparameters_2 = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.7,
    "top_k": 80,
    "top_p": 0.9,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}

# Creative Sampling
hyperparameters_3 = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 1.0,
    "top_k": 100,
    "top_p": 0.85,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}

hyperparameters_list = [hyperparameters_1, hyperparameters_2, hyperparameters_3]

In [6]:
import json
import torch
from torch.utils.data import Dataset, DataLoader

# 定义自定义数据集类
class QuestionDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['question']
        options = item['options']
        combined_question = question
        for option in options:
            combined_question += f"\n{option}"
        answer = item['correct']
        return combined_question, answer

# 读取数据集
with open(file_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# 创建数据集
data = data[:20]
question_dataset = QuestionDataset(data)

# 创建数据加载器
data_loader = DataLoader(question_dataset, batch_size=4, shuffle=True)



# 查看数据集中的一些样本
for i, (questions, answers) in enumerate(data_loader):
    print(f"Batch {i+1}:")
    for question, answer in zip(questions, answers):
        print(f"Question: {question}")
        print(f"Correct answer: {answer}")
    if i >= 1:  # 查看前两个批次
        break

Batch 1:
Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?
A)21
B)21.5
C)22
D)22.5
E)23
Correct answer: E
Question: There are k-2 members in a certain band, including Jim and Ellen. Two members are to be selected to attend the Grammy awards ceremony. If there are 6 possible combinations in which Jim and Ellen are not selected, what is the value of k?
A)8
B)9
C)10
D)11
E)12
Correct answer: A
Question: Rs. 5600 is divided into three parts A, B and C. How much A is more than C if their ratio is 1/7:1/7:1/14?
A)300
B)992
C)1120
D)552
E)312
Correct answer: C
Question: q is a positive integer and multiple of 2; p = 4^q, what is the remainder when p is divided by 10?
A)10
B)6
C)4
D)0
E)It Cannot Be Determined
Correct answer: B
Batch 2:
Question: Carl is facing very difficult financial times and can only pay th

In [7]:
# 准确率计算函数
def calculate_accuracy(predictions, references):
    correct = 0
    total = len(references)
    for pred, ref in zip(predictions, references):
        if pred == ref:
            correct += 1
    return correct / total

In [8]:
def trim(s: str, i: int) -> str:
    """
    extract the ith inference
    input:
        s: string, input
        i: the number of inference you want to extract

    output:
        string, the ith sentence
    """

    inference = s.split("Let's think step by step.\n")[2]
    result = inference.split(f"{i}. ")[1]
    result = result.split("\n")[0]
    return result

In [9]:
shot_question = "question: What is the least value of x, So that 2x5475 is divisible by 9? "
shot_option = "options: A)7, B)8, C)4, D)3, E)2\n"
shot_prompt = "Let's think step by step.\n"
shot_reason_1 = "1. The sum of the digits of the number is divisible by 9. Then the number is divisible by 9.\n"
shot_reason_2 = "2. We have 2 + x + 5 + 4 + 7 + 5 = 23 + x.\n"
shot_reason_3 = "3. The number after 23 and divisible by 9 is 27.\n"
shot_reason_4 = "4. Least value of x may be '4', so that the total 23 + 4 = 27.\n"
shot_answer = "5. Answer: C.\n"
one_shot_example = shot_question + shot_option + shot_prompt + shot_reason_1 + shot_reason_2 + shot_reason_3 + shot_reason_4 + shot_answer

In [11]:
def extract_answer(input: str, mode: bool):
    """
    extract answer from given input
    if mode == True: extract letter
    if mode == False: extract number
    """
    if mode == False:
        return re.findall(r'\d+', input)[-1]
    elif mode == True:
        return re.findall(r'\b[A-E]\b', input)[-1]

In [12]:
def multi_vote_answer(answers: list):
    return max(set(answers), key=answers.count)

In [13]:
def select(background: str, infs: list) -> int:
    """
    select one from infs
    """

    hyper = {
        "do_sample": False,
        "max_new_tokens": 5,
        "num_beams": 10,
        "num_return_sequences": 1,
        "pad_token_id": pad_token_id
    }

    prompt = "Given background above, which of following is the next most reasonable inference? "
    for i in range(len(infs)):
        prompt += f"#{i+1}: {infs[i]}\n"

    prompt += "Answer: #"
    input = background + prompt
    output = pipe(input, **hyper)
    output = output[0]['generated_text']
    ans = extract_answer(output, False)
    ans = int(ans)
    if (ans - 1) < 0 or ans >= len(infs):   # invalid selection
        return -1

    return ans - 1

In [14]:
def inference(input: str, hyper_list) -> list:
    """
    input: prompt + question

    return: list of answer for each path
    """
    i = 1
    current_inf = input
    infs = []
    answers = []
    delete = []

    while i < 8 and len(answers) < 3:                                   # most 8 steps
        for j in range(len(hyper_list)):
            if j in delete:
                continue
            try:
                output = pipe(current_inf, **(hyper_list[j]))     # infer in conservative, middle and creative mode and add them to infs
                output = output[0]['generated_text']
                inf = trim(output, i)
                print(f"Hyparams{j}:{inf}")
                if "Answer" in inf:                 # find answer, add to "answers" list and remove from hyper_list
                    ans = extract_answer(inf, True)
                    answers.append(ans)
                    delete.append(j)
                    #return ans
                else:
                    infs.append(inf)
            except:
                print(f"Hyparams{j}: Error")
                delete.append(j)

        if len(delete) == 3:
            break

        background = current_inf.split(shot_answer)[1]               # background doesn't contain shot example
        choose = select(background, infs)
        if choose == -1 or choose >= len(infs):                                            # if no valid answer, choose the first one
            choose = 0

        current_inf += f"{i}. {infs[choose]}\n"                       # add the choosen inference to current inf
        i += 1
        infs.clear()
        
    return answers

In [ ]:
predictions = []
for i, (questions, answers) in enumerate(data_loader):
    questions = list(questions)
    answers = list(answers)
    for question, answer in zip(questions, answers):
        try:
            input = f"{one_shot_example} question: {question}. {shot_prompt}"
            print(f"correct answer is {answer}")
            answers = inference(input,copy.deepcopy(hyperparameters_list))
            answer = multi_vote_answer(answers)
            print(f"answer: {answer}")
            if answer:
                predictions.append(answer)
                print(f"Predicted Answer: {answer}")
            else:
                predictions.append(None)  # 如果没有找到大写字母，则预测为空
                print("Predicted Answer: None")
        except :
            predictions.append(None)  # 如果没有找到大写字母，则预测为空
            print("Predicted Answer: None")

correct answer is E
Hyparams0:The total distance of the trail is 43 km.
Hyparams1:Friend P starts 15% faster than Friend Q.
Hyparams2:They start at the opposite ends of the trail at the same time.
Hyparams0:Friend P's rate is 15% faster than Friend Q's.
Hyparams1:Friend P's rate is 15% faster than Friend Q's, so P's rate is 1.15 times Q's rate.
Hyparams2:Friend P's rate is 15% faster than Friend Q's.
Hyparams0:Friend P starts at the opposite end of the trail.
Hyparams1:Let's say that Friend P has walked x km and Friend Q has walked (43 - x) km.
Hyparams2:If Friend P has walked 43 km/15% = 28.28 km, then Friend Q will have walked 28.28 km/15% = 19.38 km.
Hyparams0:Friend Q starts at the same end of the trail.
Hyparams1:Friend P will have walked 43 km/15% = 29.17 km when he passes Friend Q.
Hyparams2:Friend P will have walked 43/15 \* 43 = 30 km when Friend P and Q pass each other.
Hyparams0:When they pass each other, Friend P will have walked 43/1.15 = 37.9 km.
Hyparams1:The time taken 

In [ ]:
# 计算准确率
accuracy = calculate_accuracy(predictions, answers)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

In [ ]:
# 打印一些样本数据
print("\nSample Outputs:")
for q, a, p in zip(questions[:15], answers[:15], predictions[:15]):
    print(f"Question: {q}")
    print(f"True Answer: {a}")
    print(f"Predicted Answer: {p}")
    print()